# Tests de Modèles - House Prices Prediction
## Laplace Immo - Projet Data Science

Ce notebook compare différents algorithmes de Machine Learning pour prédire les prix des maisons.
Objectif: Identifier le meilleur modèle pour la prédiction des prix.

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

# Chargement des données
train_df = pd.read_csv('../data/raw/train.csv')
print(f"Données chargées: {train_df.shape}")

## 1. Prétraitement des Données

In [ ]:
print("=== PRÉTRAITEMENT DES DONNÉES ===")

# Ingénierie des features
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']
train_df['TotalSF'] = train_df['GrLivArea'] + train_df['TotalBsmtSF'].fillna(0)
train_df['OverallScore'] = train_df['OverallQual'] * train_df['OverallCond']

# Séparation features/target
X = train_df.drop(['SalePrice', 'Id'], axis=1)
y = train_df['SalePrice']

# Identification des types de variables
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Features numériques: {len(numerical_features)}")
print(f"Features catégorielles: {len(categorical_features)}")

# Gestion des valeurs manquantes
for col in numerical_features:
    X[col] = X[col].fillna(X[col].median())

for col in categorical_features:
    X[col] = X[col].fillna('None')

print("Valeurs manquantes traitées")

## 2. Pipeline de Prétraitement

In [ ]:
print("=== PIPELINE DE PRÉTRAITEMENT ===")

# Pipeline pour les variables numériques
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline pour les variables catégorielles
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine les pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Jeu d'entraînement: {X_train.shape}")
print(f"Jeu de test: {X_test.shape}")

# Application du prétraitement
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
print(f"Données prétraitées: {X_train_processed.shape}")

## 3. Tests des Modèles

In [ ]:
print("=== TESTS DES MODÈLES ===")

# Dictionnaire des modèles à tester
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Fonction d'évaluation
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Entraîne et évalue un modèle"""
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Métriques
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    return {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'cv_rmse': cv_rmse
    }

# Évaluation de tous les modèles
results = {}
for name, model in models.items():
    print(f"\nÉvaluation de {name}...")
    results[name] = evaluate_model(model, X_train_processed, y_train, X_test_processed, y_test)
    print(f"  RMSE Test: {results[name]['test_rmse']:,.0f}")
    print(f"  R² Test: {results[name]['test_r2']:.3f}")
    print(f"  CV RMSE: {results[name]['cv_rmse']:,.0f}")

## 4. Visualisation des Résultats

In [ ]:
print("=== VISUALISATION DES RÉSULTATS ===")

# Création du DataFrame de résultats
results_df = pd.DataFrame(results).T
print("\nTableau comparatif des modèles:")
print(results_df.round(3))

# Graphique comparatif
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# RMSE Test
axes[0, 0].bar(results_df.index, results_df['test_rmse'], color='skyblue', alpha=0.7)
axes[0, 0].set_title('RMSE Test par Modèle')
axes[0, 0].set_ylabel('RMSE ($)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# R² Test
axes[0, 1].bar(results_df.index, results_df['test_r2'], color='lightgreen', alpha=0.7)
axes[0, 1].set_title('R² Test par Modèle')
axes[0, 1].set_ylabel('R² Score')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# MAE Test
axes[1, 0].bar(results_df.index, results_df['test_mae'], color='salmon', alpha=0.7)
axes[1, 0].set_title('MAE Test par Modèle')
axes[1, 0].set_ylabel('MAE ($)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# CV RMSE
axes[1, 1].bar(results_df.index, results_df['cv_rmse'], color='gold', alpha=0.7)
axes[1, 1].set_title('CV RMSE par Modèle')
axes[1, 1].set_ylabel('CV RMSE ($)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Identification du Meilleur Modèle

In [ ]:
# Identification du meilleur modèle
best_model_name = results_df['test_r2'].idxmax()
best_r2 = results_df.loc[best_model_name, 'test_r2']
best_rmse = results_df.loc[best_model_name, 'test_rmse']
best_cv_rmse = results_df.loc[best_model_name, 'cv_rmse']

print(f"\n🏆 MEILLEUR MODÈLE: {best_model_name}")
print(f"   R² Score: {best_r2:.3f}")
print(f"   RMSE: ${best_rmse:,.0f}")
print(f"   CV RMSE: ${best_cv_rmse:,.0f}")

# Analyse des résidus du meilleur modèle
best_model = models[best_model_name]
y_pred_test = best_model.predict(X_test_processed)

plt.figure(figsize=(12, 5))

# Résidus vs Valeurs prédites
plt.subplot(1, 2, 1)
residuals = y_test - y_pred_test
plt.scatter(y_pred_test, residuals, alpha=0.5, color='steelblue')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.title(f'Résidus - {best_model_name}')
plt.grid(True, alpha=0.3)

# Distribution des résidus
plt.subplot(1, 2, 2)
plt.hist(residuals, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.title(f'Distribution des résidus - {best_model_name}')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/best_model_residuals.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nÉtude des résidus:")
print(f"Moyenne des résidus: {residuals.mean():.0f}")
print(f"Écart-type des résidus: {residuals.std():.0f}")

## 6. Optimisation du Meilleur Modèle

In [ ]:
print("=== OPTIMISATION DU MEILLEUR MODÈLE ===")

# Optimisation des hyperparamètres pour Gradient Boosting
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01],
    'subsample': [0.8, 0.9, 1.0]
}

print("Recherche des meilleurs hyperparamètres...")
grid_search = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_processed, y_train)

print(f"Meilleurs paramètres: {grid_search.best_params_}")
print(f"Meilleur score CV: {-grid_search.best_score_:,.0f}")

# Évaluation du modèle optimisé
best_model_optimized = grid_search.best_estimator_
y_pred_optimized = best_model_optimized.predict(X_test_processed)

optimized_rmse = np.sqrt(mean_squared_error(y_test, y_pred_optimized))
optimized_r2 = r2_score(y_test, y_pred_optimized)
optimized_mae = mean_absolute_error(y_test, y_pred_optimized)

print(f"\nRésultats du modèle optimisé:")
print(f"RMSE: ${optimized_rmse:,.0f}")
print(f"R²: {optimized_r2:.3f}")
print(f"MAE: ${optimized_mae:,.0f}")

# Comparaison avant/après optimisation
print(f"\nAmélioration:")
print(f"R²: {optimized_r2 - best_r2:.3f} points de gain")
print(f"RMSE: ${best_rmse - optimized_rmse:,.0f} de réduction")

## 7. Sauvegarde du Modèle Final

In [ ]:
import joblib

print("=== SAUVEGARDE DU MODÈLE FINAL ===")

# Sauvegarde du modèle et du préprocesseur
model_final = {
    'model': best_model_optimized,
    'preprocessor': preprocessor,
    'model_name': best_model_name,
    'metrics': {
        'rmse': optimized_rmse,
        'r2': optimized_r2,
        'mae': optimized_mae
    }
}

# Créer le répertoire des modèles s'il n'existe pas
import os
os.makedirs('../models', exist_ok=True)

# Sauvegarde
joblib.dump(model_final, '../models/house_prices_model.pkl')
print("Modèle final sauvegardé dans: ../models/house_prices_model.pkl")

# Sauvegarde des résultats
results_df.to_csv('../reports/model_comparison_results.csv')
print("Résultats sauvegardés dans: ../reports/model_comparison_results.csv")

# Sauvegarde des meilleurs paramètres
with open('../reports/best_params.txt', 'w') as f:
    f.write(f"Meilleur modèle: {best_model_name}\n")
    f.write(f"Meilleurs paramètres: {grid_search.best_params_}\n")
    f.write(f"R² Score: {optimized_r2:.3f}\n")
    f.write(f"RMSE: ${optimized_rmse:,.0f}\n")
    f.write(f"MAE: ${optimized_mae:,.0f}\n")

print("Paramètres sauvegardés dans: ../reports/best_params.txt")

# Résumé final
print("\n" + "="*60)
print("RÉSUMÉ FINAL DU PROJET HOUSE PRICES PREDICTION")
print("="*60)
print(f"Modèle retenu: {best_model_name}")
print(f"Performance R²: {optimized_r2:.1%}")
print(f"Erreur moyenne (RMSE): ${optimized_rmse:,.0f}")
print(f"Erreur absolue moyenne (MAE): ${optimized_mae:,.0f}")
print("\nLe modèle est prêt pour le déploiement en production!")